In [ ]:
print("Python Settings (Ok)")

In [ ]:
from git import Repo
import os
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

#!pip install -q langchain_community 
from langchain_community.llms import Ollama

In [ ]:
%pwd

In [ ]:
!mkdir test_repo

In [ ]:
repo_path = "test_repo/"
repo = Repo.clone_from("https://github.com/ram541619/CustomerOnboardFlow", to_path=repo_path)
print (repo)
print (repo_path)

In [ ]:
import re
import os

def find_dotnet_core_apis(directory_path):
    api_routes = []
    # Regular expressions to match route attributes and HTTP methods
    route_pattern = r'^\s*\[Route\("([^\)]+)"\)\]'
    http_method_pattern = r"\[Http(Get|Post|Put|Delete|Patch|Options|Head)\s*(?:\((.*?)\))?\]"

    # Walk through all the .cs files in the directory
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            if file.endswith(".cs"):
                file_path = os.path.join(root, file)
                with open(file_path, 'r', encoding='utf-8') as f:
                    lines = f.readlines()
                    # Check for API routes and HTTP methods in the file
                    current_route = None
                    for line in lines:
                        route_match = re.match(route_pattern, line.strip())
                        if route_match:
                            # Extract the base route of the controller
                            current_route = route_match.group(1)
                        
                        http_method_match = re.match(http_method_pattern, line.strip())
                        if http_method_match:
                            # Extract the HTTP method (e.g., GET, POST, etc.)
                            http_method = http_method_match.group(1).strip()
                            route_suffix = http_method_match.group(2)  # Optional route parameter
                            api_route = current_route
                            if route_suffix:
                                api_route += f"/{route_suffix}"
                            api_route += f" [{http_method.upper()}]"
                            api_routes.append(api_route)
    return api_routes

# Example usage
directory_path = repo_path 
api_routes = find_dotnet_core_apis(directory_path)
for route in api_routes:
    print(route)

In [ ]:
from langchain_community.llms import Ollama

def generate_bdd_tests(api_routes):
    # Initialize the Ollama model
    model = Ollama(model="llama3.2:1B") 
    
    # Create BDD-style test generation prompt
    prompt = f"""
    Generate BDD-style (Gherkin syntax) test cases for the following API routes. 
    The API routes are as follows:
    {api_routes}
    The tests should cover different HTTP methods like GET, POST, PUT, DELETE, and edge cases.
    """

    # Generate the response using Ollama
    response = model.invoke(input=prompt)

    return response.strip()

bdd_tests= generate_bdd_tests(api_routes)
print(bdd_tests)